In [1]:
import duckdb
import pandas as pd

In [2]:
# 체류데이터 확인
con = duckdb.connect()
con.execute("select * from read_parquet('../_data/0612_반출자료/202407_metropolitan_duration_time.parquet') limit 10").df()

,operation_date,station_id,cluster_id,admin_code,station_name,station_type,duration_type,total_duration_time,duration_count,duration_average_time,duration_median_time
0,20240725,2546,CG6375150378,1121510300,아차산(어린이대공원후문),T,play,1.730365e+05,1574.0,109.934273,74.191666
1,20240727,70761,CG5057841452,1162010200,서울산업정보학교.삼성교,B,work,1.215279e+04,49.0,248.016159,249.633331
2,20240724,72449,CG5503452241,1111015500,청계2가.수표교,B,work,3.660750e+02,2.0,183.037506,183.037506
3,20240723,2800684,CG2684045350,2826011000,대은교회,B,work,3.340308e+03,14.0,238.593445,242.066666
4,20240717,4114363,CG5551818642,4111313100,수원역환승센터(7승강장),B,work,2.296378e+04,100.0,229.637756,251.970825
5,20240715,4105578,CG6091870932,4115010400,신곡교,B,other,1.723143e+04,49.0,351.661896,326.366669
6,20240720,222,CG5828544420,1168010100,강남,T,play,3.078486e+06,25174.0,122.288292,100.929169
7,20240710,4204,CG4689853247,1144012700,디지털미디어시티,T,work,2.423959e+06,8435.0,287.369171,290.200012
8,20240715,4102025,CG4313525557,4127110700,부곡제2종합시장,B,live,1.278362e+04,32.0,399.488281,392.041687
9,20240702,9406,CG4834855675,1138010800,응암역신사오거리,B,live,1.253255e+05,334.0,375.226196,386.787476


In [19]:
import holidays

# Initialize Korean holidays
korea_holidays = holidays.country_holidays('KR', years=[2025]) # KR is the country code for South Korea
korea_holidays

{datetime.date(2025, 1, 1): "New Year's Day", datetime.date(2025, 1, 29): 'Korean New Year', datetime.date(2025, 1, 28): 'The day preceding Korean New Year', datetime.date(2025, 1, 30): 'The second day of Korean New Year', datetime.date(2025, 3, 1): 'Independence Movement Day', datetime.date(2025, 5, 5): "Buddha's Birthday; Children's Day", datetime.date(2025, 6, 6): 'Memorial Day', datetime.date(2025, 8, 15): 'Liberation Day', datetime.date(2025, 10, 3): 'National Foundation Day', datetime.date(2025, 10, 9): 'Hangul Day', datetime.date(2025, 10, 6): 'Chuseok', datetime.date(2025, 10, 5): 'The day preceding Chuseok', datetime.date(2025, 10, 7): 'The second day of Chuseok', datetime.date(2025, 12, 25): 'Christmas Day', datetime.date(2025, 3, 3): 'Alternative holiday for Independence Movement Day', datetime.date(2025, 5, 6): "Alternative holiday for Buddha's Birthday; Alternative holiday for Children's Day", datetime.date(2025, 10, 8): 'Alternative holiday for Chuseok', datetime.date(202

In [105]:
# 일별/시간대별 집계 팡리
df = con.execute("""SELECT count(승차정류장명칭)/count(*)
        FROM read_parquet('../_data/202502_workod_purpose_transport_grid.parquet')
        limit 10""").df()
df

,"(count(""승차정류장명칭"") / count_star())"
0,0.969915


In [96]:
df = con.execute("""SELECT distinct(trip_type)
    FROM read_parquet('../_output/2024-07-07_agg_result_organized.parquet')
    """).df()
df # PK로 잘 적용됨

,trip_type
0,승차
1,하차


In [ ]:
# 월별 평일 수 : 1-18 2-20 3-20 4-22 5-20 6 -18

In [55]:
con = duckdb.connect()
monthly_normal_day = {'202501':18, '202502':20, '202503':20, '202504':22, '202505':20, '202506':18}

for month, normal_days in monthly_normal_day.items():
    print(month, normal_days)

202501 18
202502 20
202503 20
202504 22
202505 20
202506 18


In [72]:

for month, normal_days in monthly_normal_day.items():
    con.execute(f"""COPY (SELECT '{month}' AS standard_ym
                        , 승차정류장ID AS departure_station_id
                        , 승차정류장명칭 AS departure_station_name
                        , 승차지역코드 AS departure_region_code
                        , 승차교통수단구분 AS departure_station_type
                        , 승차그리드ID AS departure_grid_id


                        , 하차정류장ID AS arrival_station_id
                        , 하차정류장명칭 AS arrival_station_name
                        , 하차지역코드 AS arrival_region_code
                        , 하차교통수단구분 AS arrival_station_type
                        , 하차그리드ID AS arrival_grid_id

                        , CASE WHEN 출퇴근구분 ='morning'  THEN '출근'
                                WHEN 출퇴근구분 ='evening' THEN '퇴근' END AS purpose_name
                        , ROUND(월_총_통행량/{normal_days}, 1) AS daily_use_count
                        , 탑승시간_중위 AS median_elapse_time
                        , 탑승시간_평균 AS average_elapse_time
                        , 이동거리_중위 AS median_distance
                        , 이동거리_평균 AS average_distance
                        , 승차_5179 AS departure_station_geometry
                        , 하차_5179 AS arrival_station_geometry
    FROM read_parquet('../_output/{month}_workod_purpose_transport_grid.parquet')
    ) TO '../_output/{month}_workod_purpose_transport_organized.parquet' """)
    print(f"{month} db업로드 전 마지막 테이블 컬럼 정제 완료")
df # PK로 잘 적용됨

202501 db업로드 전 마지막 테이블 컬럼 정제 완료
202502 db업로드 전 마지막 테이블 컬럼 정제 완료
202503 db업로드 전 마지막 테이블 컬럼 정제 완료
202504 db업로드 전 마지막 테이블 컬럼 정제 완료
202505 db업로드 전 마지막 테이블 컬럼 정제 완료
202506 db업로드 전 마지막 테이블 컬럼 정제 완료


,dt,승차정류장ID,하차정류장ID,승하차구분,hour,승차지역코드,하차지역코드,승차교통수단구분,하차교통수단구분,통행량,...,평균탑승거리,중위탑승시간,평균환승건수,중위환승건수,승차정류장명칭,승차_gridid,하차정류장명칭,하차_gridid,승차_grid_id,하차_grid_id
0,20240707,72063,기타,승차,11,11100,None,B,None,10.0,...,1089.7,1229.5,0.3,0.0,장곡초교사거리,CG6028757618,기타,None,POINT (960287 1957618),None


In [90]:
# 일별/시간대별 집계 파일
days = ["2024-07-07"]
for day in days:
    con.execute(f"""COPY (SELECT STRPTIME(CAST(dt AS VARCHAR), '%Y%m%d') AS standard_date
                        , hour
                        , COALESCE(승차정류장ID, '기타') AS departure_station_id
                        , COALESCE(승차정류장명칭, '기타') AS departure_station_name
                        , COALESCE(승차지역코드, '기타') AS departure_region_code
                        , COALESCE(승차교통수단구분, '기타') AS departure_station_type
                        , COALESCE(승차_gridid, '기타') AS departure_grid_id


                        , COALESCE(하차정류장ID, '기타') AS arrival_station_id
                        , COALESCE(하차정류장명칭, '기타') AS arrival_station_name
                        , COALESCE(하차지역코드, '기타') AS arrival_region_code
                        , COALESCE(하차교통수단구분, '기타') AS arrival_station_type
                        , COALESCE(하차_gridid, '기타') AS arrival_grid_id

                        , CASE WHEN 승하차구분 IN ('board', '승차') THEN '승차'
                                WHEN 승하차구분 IN ('alight', '하차') THEN '하차' END AS trip_type
                        , 통행량 AS use_count
                        , 통행명수 AS unique_use_count
                        , 중위탑승시간 AS median_elapse_time
                        , 평균탑승거리 AS average_elapse_time
                        , 중위이동거리 AS median_distance
                        , 평균이동거리 AS average_distance
                        , 중위환승건수 AS median_transfer_count
                        , 평균환승건수 AS average_transfer_count
                        , 승차_5179 AS departure_station_geometry
                        , 하차_5179 AS arrival_station_geometry
    FROM read_parquet('../_output/{day}_agg_result_coordinates.parquet')
    ) TO '../_output/{day}_agg_result_organized.parquet' """)
    print(f"{day} db업로드 전 마지막 테이블 컬럼 정제 완료")
 # PK로 잘 적용됨

2024-07-07 db업로드 전 마지막 테이블 컬럼 정제 완료
